In [ ]:
pip install azure-ai-formrecognizer azure-search-documents openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import os

Azure AI services | Document intelligence -
FormRecognizer resource

In [ ]:
# Set up the Form Recognizer client
endpoint = "https://.cognitiveservices.azure.com/"
api_key = ""
doc_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(api_key))


In [ ]:
import os
os.listdir()

['.config', 'contract.pdf', 'sample_data']

In [ ]:
clause_titles = ["Price Protection", "Late Payment Penalties", "SLAs/Service Credits", "Termination", "Confidentiality", "Indemnification", "Rebates/Volume Discounts", "Payment Terms", "Forex", "COLA/Inflation Adjustment", "Early Payment Discounts", "Early Payment Discounts", "Termination Provisions", "Committal/Non-Committal Contracts"]


In [ ]:
def extract_clauses(pdf_path):
    with open(pdf_path, "rb") as fd:
        poller = doc_client.begin_analyze_document("prebuilt-layout", fd)
        result = poller.result()

    # Extract text content
    '''
    extracted_text = ""
    for page in result.pages:
        for line in page.lines:
            extracted_text += line.content + "\n"
    return extracted_text
    '''

    clause_segments = []
    current_label = None
    current_text = []

    for page in result.pages:
        for line in page.lines:
            line_text = line.content.strip()

            # Check if paragraph text matches any clause title
            matched_title = next((title for title in clause_titles if title.lower() in line_text.lower()), None)

            if matched_title:
                # Save previous clause segment if there's any
                if current_label and current_text:
                    clause_segments.append({
                        "Label": current_label,
                        "Text": " ".join(current_text)
                    })

                # Reset for new clause
                current_label = matched_title
                current_text = [line_text]
            else:
                # Continue accumulating text for the current clause
                current_text.append(line_text)

    # Add the final clause to segments if it exists
    if current_label and current_text:
        clause_segments.append({
            "Label": current_label,
            "Text": " ".join(current_text)
        })

    return clause_segments

# Call the function to process the contract and get clause segments
file_path = "contract.pdf"
clause_segments = extract_clauses(file_path)

# Check the output format
for clause in clause_segments:
    print(f"Label: {clause['Label']}, Text: {clause['Text'][:100]}...")  # Display first 100 characters of each clause


Label: Rebates/Volume Discounts, Text: 1. Rebates/Volume Discounts The Supplier agrees to provide volume-based discounts to the Buyer, ince...
Label: SLAs/Service Credits, Text: 2. SLAs/Service Credits The Supplier commits to maintaining a minimum service level, as outlined in ...
Label: Price Protection, Text: 3. Price Protection The prices set forth in this agreement shall remain fixed for the initial 12 mon...
Label: Forex, Text: 4. Forex The exchange rate applied to payments under this contract shall be locked at the rate in ef...
Label: COLA/Inflation Adjustment, Text: 5. COLA/Inflation Adjustment The contract pricing will be reviewed and adjusted annually based on th...
Label: Payment Terms, Text: 6. Payment Terms The Buyer agrees to settle all invoices within 30 days of receipt. Payments will be...
Label: Early Payment Discounts, Text: 7. Early Payment Discounts The Buyer is entitled to a 2% discount on the total invoice amount if ful...
Label: Late Payment Penalties, Text: 8. L

In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


In [ ]:
import re
import openai
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField



Cognitive Search -> Azure AI services | AI Search - Search Service resouce.
Also manually creae an index with required fields

In [ ]:
search_endpoint = "https://.search.windows.net"
search_api_key = ""
index_name = "contind"
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=AzureKeyCredential(search_api_key))


In [ ]:
def create_or_update_index(clause_segments):
    # Create a simple index for storing clause titles and content
    documents = [
        {
            "id": f"clause-{i}",  # Unique ID for each clause
            "title": clause["Label"],
            "text": clause["Text"]
        }
        for i, clause in enumerate(clause_segments)
    ]
    # Upload the document
    search_client.upload_documents(documents=documents)

# Index the extracted contract text
create_or_update_index(clause_segments)

In [ ]:
# Query all documents in the index
def get_all_documents():
    results = search_client.search(search_text="*", top=10)  # Use "*" to match all documents
    for result in results:
        print(result)

# Call the function to print documents
get_all_documents()

{'title': 'SLAs/Service Credits', 'id': 'clause-1', 'text': "2. SLAs/Service Credits The Supplier commits to maintaining a minimum service level, as outlined in the Service Level Agreement (SLA) Schedule of this contract. This includes an uptime percentage of no less than 99.5% monthly. In the event of any service interruptions or downtimes exceeding the allowable limit, the Supplier will provide service credits to the Buyer based on the duration and impact of the downtime. Service credits will be calculated as a percentage of the monthly service fee, with specific percentages detailed in the SLA Schedule. Credits will be applied to the Buyer's next billing cycle. If the Supplier fails to meet the SLA for three consecutive months, the Buyer reserves the right to review the agreement and, if necessary, terminate the contract without penalty. Both parties agree that the Supplier will proactively report service performance at the end of each month, providing transparency and accountabilit

In [ ]:
def search_for_clause(clause_title):
    results = search_client.search(clause_title)  # Search for a specific clause title
    clauses = []
    for result in results:
        clauses.append(result['text'])
    return clauses

# Search for a specific clause
clause_title = "Payment Terms"
clauses = search_for_clause(clause_title)
print(clauses)

["6. Payment Terms The Buyer agrees to settle all invoices within 30 days of receipt. Payments will be made via bank transfer to the Supplier's designated account. In exchange for early payment, the Supplier offers a 2% discount if the invoice is paid within 10 days. This clause incentivizes the Buyer to make early payments while offering a standard 30-day payment period. The early payment discount supports the Supplier's cash flow while providing savings for the Buyer.", "Company A Contract Document\nContract Title: Supplier Agreement\nThis Agreement is made and entered into as of [Date], by and between:\nSupplier Name: [Supplier A]\nBuyer Name: [Buyer A]\nContract Duration: [Start Date] to [End Date]\n1. Rebates/Volume Discounts\nThe Supplier agrees to provide volume-based discounts to the Buyer, incentivizing higher\npurchase volumes with proportional rebates. For each calendar quarter, the Supplier will\nassess the total purchase volume made by the Buyer and apply tiered discounts 

In [ ]:
!pip install openai

In [ ]:
import openai
import os

Create Azure OpenAI resource - GPT-4 deployment

Test env - GPT-35 deployment with its own resource

In [ ]:
from openai import AzureOpenAI
client = AzureOpenAI(
  api_key="",
  api_version = "2024-02-01",
  azure_endpoint = "https://-m3bvvhbw-westeurope.openai.azure.com/"
)


In [ ]:

def summarize_clause(clause_text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant specialized in summarizing legal clauses."},
        {"role": "user", "content": f"Please summarize the following clause: {clause_text}"}
    ]

    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )
    summary = response.choices[0].message.content.strip()
    return summary

In [ ]:
def generate_summary(clause_title, relevant_clauses):
    prompt = f"Summarize the information about '{clause_title}' using the following references: {relevant_clauses}. If the information doesn't cover the clause title, indicate that it is absent."
    messages=[
            {"role": "system", "content": "You are a contract summarization assistant."},
            {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )
    summary = response.choices[0].message.content.strip()
    return summary


In [ ]:
# List of clause titles you are looking for
'''
clause_titles = [
    "Termination Provisions",
    "Confidentiality",
    "Indemnification",
    "Rebates/Volume Discounts",
    "Payment Terms",
    "SLAs/Service Credits",
    "Price Protection",
    "Forex",
    "COLA/Inflation Adjustment",
    "Late Payment Penalties"
]
'''
clause_titles = [
    "Price Protection",
    "Termination Provisions",
    "Invalid"
]

def generate_report(clause_titles):
    report = ""
    for title in clause_titles:
        clauses = search_for_clause(title)
        if clauses:
            report += f"Clause: {title}\n"
            summary = generate_summary(title, clauses)
            report += f"Summary: {summary}\n\n"
            '''
            for clause in clauses:
                summary = summarize_clause(clause)
                report += f"Summary: {summary}\n\n"
            '''
        else:
            report += f"Clause: {title} - Absent\n\n"
    return report

# Generate and print the final report
final_report = generate_report(clause_titles)
print(final_report)


Clause: Price Protection
Summary: The information provided under the reference '3. Price Protection' states that the prices set forth in the agreement will remain fixed for the initial 12 months, and thereafter, pricing will be subject to an annual review based on market indices for comparable goods and services, with any price adjustment limited to a maximum increase of 3% per annum unless agreed otherwise. The clause also allows the buyer to request a formal pricing review in the event of significant market fluctuations and requires the Supplier to provide a detailed breakdown of cost factors justifying any proposed price increase exceeding the agreed limit. Both parties agree to meet and negotiate in good faith to resolve any pricing disputes, with the goal of maintaining a fair and stable pricing structure throughout the contract term. The information given under other

Clause: Termination Provisions
Summary: The Termination Provisions clause allows either party to end the contract

In the same azure OpenAI resource with gpt4/35 deployment create test-embedding-ada-002 deployment

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
def generate_embeddings(text):
    response = client.embeddings.create(
        input=text, model="text-embedding-ada-002"
    )
    embeddings = [embedding.embedding for embedding in response.data]  # Correct way to access the embedding

    return embeddings

In [ ]:
def retrieve_relevant_clause(question, clause_segments):
    # Generate embeddings for the question
    question_embedding = generate_embeddings([question])[0]

    # Generate embeddings for all clauses
    clause_embeddings = generate_embeddings([clause['Text'] for clause in clause_segments])

    # Compute cosine similarity between question embedding and clause embeddings
    similarities = cosine_similarity([question_embedding], clause_embeddings)

    # Find the most relevant clause based on the highest similarity score
    most_relevant_idx = np.argmax(similarities)

    return clause_segments[most_relevant_idx]


In [ ]:

# Step 3: Use the retrieved clause to answer the question
def answer_question(question, relevant_clause):
    prompt = f"Given the following contract clause, answer the question: '{question}'\n\nClause: {relevant_clause['Text']}\n\nAnswer:"
    messages=[
            {"role": "system", "content": "You are a legal assistant."},
            {"role": "user", "content": prompt}
        ]
    # Call Azure OpenAI API to generate an answer
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )

    return response.choices[0].message.content.strip()


In [ ]:

# Example question about the contract
question = "What percent interest do I pay for late penalties?"

# Retrieve relevant clause
relevant_clause = retrieve_relevant_clause(question, clause_segments)

# Get the answer for the question
answer = answer_question(question, relevant_clause)

print(f"Answer: {answer}")

Answer: You pay 1.5% interest per month as late penalties.
